In [1]:
#Dependencies 
import pandas as pd

In [2]:
#create variable for path of file we will open
filepath = 'data/purchase_data.csv'

In [3]:
df = pd.read_csv(filepath) #create df from csv file


#  Player Count

Display the total number of players



In [4]:
num_players = df['SN'].nunique() #count the number of unique players in the SN column 

player_count = pd.DataFrame({'Total Players' : num_players}, index=[0]) #create df with above data


 # Purchasing Analysis (Total)
 

In [5]:
num_items = df['Item ID'].nunique() #find number of unique items by item ID

In [6]:
total_revenue = df['Price'].sum() #calculate total revenue

In [7]:
num_purchase = df['Purchase ID'].count() #calculate number of purchases

In [8]:
df_price = df.drop_duplicates(subset='Item ID')
avg_price = df_price.mean()['Price']
avg_price = round(avg_price, 2)

In [9]:
stats_dictionary = {'Number of Unique Items' : num_items, 'Average Price' : avg_price, 'Number of Purchases' :
                   num_purchase, 'Total Revenue': total_revenue}
pd.DataFrame(stats_dictionary, index=[0])


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.04,780,2379.77


# Gender Demographics

In [10]:
gender_count_df = df[['Gender', 'SN']] #create df with gender and SN column
gender_count_df = gender_count_df.drop_duplicates(subset='SN') #drop duplicate of SN to get unique people in a DF
gender_group = gender_count_df.groupby("Gender") 
gender_count = gender_group.count() #groupby gender
total_count = gender_count.sum() #get total number of people (this is being duplicated)
gender_percent = gender_count / total_count * 100 #create variable for count of each gender divided by the total
gender_percent = round(gender_percent, 2) #round to 2 decimal places
gender_count['Percent'] = gender_percent #add to DF created above
gender_count = gender_count.rename(columns={'SN':'Total Count'}) #rename column 
gender_count

,Total Count,Percent
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


# Purchasing Analysis (Gender)
 

In [11]:
gender_ann = df.groupby('Gender')
gender_avg_price = gender_ann['Price'].mean() #get average purchase price by gender
gender_avg_price = round(gender_avg_price, 2) #round to nearest penny
gender_purchase_count = gender_ann['Purchase ID'].count() #get number of purchases by gender
gender_total_purchase = gender_ann['Price'].sum() #get total spent by gender
gender_avg_purchase = df.groupby(['Gender', 'SN']) #group by gender, then by person (grouping by gender may not have been necessary here)
mean1 = gender_avg_purchase['Price'].sum() #get total spent by person
gender_avg_purchase = mean1.groupby("Gender").mean() #get average total spent per person, by gender
gender_avg_purchase = round(gender_avg_purchase, 2) #round to the nearest penny

In [12]:
#create DF for analysis
purchase_analysis_gender = pd.DataFrame(gender_total_purchase)
#add columns to df
purchase_analysis_gender['Average Purchase'], purchase_analysis_gender['Average Price Per Item'], purchase_analysis_gender['Number of Purchases']  = gender_avg_purchase, gender_avg_price, gender_purchase_count
#rename first column
purchase_analysis_gender = purchase_analysis_gender.rename(columns={'Price': 'Total Spent'})
purchase_analysis_gender

,Total Spent,Average Purchase,Average Price Per Item,Number of Purchases
Gender,,,,
Female,361.94,4.47,3.20,113
Male,1967.64,4.07,3.02,652
Other / Non-Disclosed,50.19,4.56,3.35,15


# Age Demographics


In [13]:
max_age = df['Age'].max() #finf max age for bins list
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, max_age] #create bins list
age_label = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+'] #create bins label list
ages = pd.DataFrame(df[['Age', 'SN']]) #create new df with only relevant data
ages = ages.drop_duplicates(subset='SN') #drop duplicate individuals/people
ages = ages.drop('SN', axis=1) #the SN column is no longer necessary
ages['Age Range'] = pd.cut(ages['Age'], age_bins, labels=age_label) #add bins column 
ages = ages.groupby('Age Range').count() #groupby each bin
ages = pd.DataFrame(ages) #convert groupby into a new df
ages = ages.rename(columns={'Age': 'Count'}) #rename column to be more descriptive 
age_percent = ages['Count']/num_players * 100 #calculate percentage
age_percent = round(age_percent, 2) #round to 2 decimal places
ages['Percentage of Players'] = age_percent #add percentage list to df
ages

,Count,Percentage of Players
Age Range,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


# Purchasing Analysis (Age)
 

In [14]:
age_purchase = df #copy df
age_purchase['Age Range'] = pd.cut(age_purchase['Age'], age_bins, labels=age_label) #create bins
age_purchase = age_purchase.groupby('Age Range') #group by the bins
purchase_by_age = age_purchase.count()['Purchase ID'] #count number of purchases in each bin
avg_buy_by_age = round(age_purchase.mean()['Price'], 2) #get average purchase price and round to nearest penny
total_purchase_age = age_purchase.sum()['Price'] #get total purchased by age

#calculate average total purchase per person
total_purchase_by_age = df #copy df
total_purchase_by_age['Age Range'] = pd.cut(total_purchase_by_age['Age'], age_bins, labels=age_label) #create bins
total_purchase_by_sn = total_purchase_by_age.groupby('SN').sum()['Price'] #get total each SN purchased
total_purchase_by_sn = pd.DataFrame(total_purchase_by_sn) #convert groupby into a df
total_purchase_by_sn = total_purchase_by_sn.rename(columns={'Price': 'Total Spent'}) #rename column to be more descriptive 
total_purchase_by_age = pd.merge(total_purchase_by_age, total_purchase_by_sn, on='SN') #merge df
total_purchase_by_age = total_purchase_by_age.drop_duplicates(subset='SN') #drop duplicate individuals/people
total_purchase_by_age = total_purchase_by_age.groupby('Age Range').mean()['Total Spent'] #groupby the bins, and get the average total spent
total_purchase_by_age = round(total_purchase_by_age, 2) #round to nearest penny

In [15]:
purchase_by_age_analysis = pd.DataFrame(purchase_by_age) #create df
purchase_by_age_analysis = purchase_by_age_analysis.rename(columns={'Purchase ID': 'Purchase Count'}) #rename column
#add columns
purchase_by_age_analysis['Average Purchase Price'], purchase_by_age_analysis['Total Purchase Value'], purchase_by_age_analysis['Average Total Purchase per Person'] = avg_buy_by_age, total_purchase_age, total_purchase_by_age
purchase_by_age_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Range,,,,
<10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


# Top Spenders

In [16]:
# total_purchase_by_sn

#spenders = df #copy df
#total_purchase_by_sn.head()

number_purchase_spender = df.groupby('SN').count()['Price'] #count number of purchases each person made
average_price_spender = round(df.groupby('SN').mean()['Price'], 2) #find average price paid for each person and round to nearest penny
total_spent_spender = df.groupby('SN').sum()['Price'] #find total spent per person
number_purchase_spender = pd.DataFrame(number_purchase_spender) #convert into DF
number_purchase_spender = number_purchase_spender.rename(columns={'Price': 'Number of Purchases'}) #rename column
average_price_spender = pd.DataFrame(average_price_spender)
average_price_spender = average_price_spender.rename(columns={'Price': 'Average Purchase Price'})
total_spent_spender = pd.DataFrame(total_spent_spender)
total_spent_spender = total_spent_spender.rename(columns={'Price': 'Total Purchase Value'})

spenders = pd.merge(number_purchase_spender, average_price_spender, on='SN') #merge number of purchase and average price df
spenders = pd.merge(spenders, total_spent_spender, on='SN')
spenders = spenders.sort_values(by='Total Purchase Value', ascending=False)
spenders.head(5)

,Number of Purchases,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


# Most Popular Items


In [17]:
#item ID, Item Name, Purchase count, Item Price, Total Purchase value
popular_item = df
popular_item = popular_item[['Item ID', 'Item Name', 'Price' ]] #create df with Item ID, Item Name and Price

popular_name = popular_item.groupby('Item ID').count()['Price'] #find how many times each item was purchased
popular_name = pd.DataFrame(popular_name) #convert groupby into df
popular_name = popular_name.rename(columns={'Price': 'Purchase Count'}) #rename column

popular_item = popular_item.drop_duplicates(subset='Item ID') #drop duplicate Item ID from df
popular_item = pd.merge(popular_item, popular_name, on='Item ID') #add purchase count column 
popular_item = popular_item.rename(columns={'Price': 'Item Price'})

popular_value = popular_item['Item Price'] * popular_item['Purchase Count'] #calculate total purchase value
popular_item['Total Purchase Value'] = popular_value #add total purchase value column to df
popular_item = popular_item.sort_values(by=['Purchase Count', 'Item ID'], ascending=False) #sort by number of times each item was purchased
popular_item.head(5)

,Item ID,Item Name,Item Price,Purchase Count,Total Purchase Value
24,178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,12,50.76
94,145,Fiery Glass Crusader,4.58,9,41.22
0,108,"Extraction, Quickblade Of Trembling Hands",3.53,9,31.77
17,82,Nirvana,4.90,9,44.10
112,103,Singed Scalpel,4.35,8,34.80


# Most Profitable Items

In [18]:
profitable_item = popular_item #copy df from most popular items
profitable_item = profitable_item.sort_values(by='Total Purchase Value', ascending=False)
profitable_item.head(5)

,Item ID,Item Name,Item Price,Purchase Count,Total Purchase Value
24,178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,12,50.76
17,82,Nirvana,4.90,9,44.10
94,145,Fiery Glass Crusader,4.58,9,41.22
2,92,Final Critic,4.88,8,39.04
112,103,Singed Scalpel,4.35,8,34.80
